<a href="https://colab.research.google.com/github/g185/AMLrepository/blob/main/RecipeNet/AML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To Do**

*   Add Params in order to start make the two datasets parametric
(Project Parameters + Data extraction & preprocessing)
*   Loading Function
(Training + Loading)
*   Example
(Example)
*   Add information about size of dataset in parameters
(Projexct Parameters)


#***Download and unzip Dataset***


*   Create folders
*   Download Recipe 5K + Annotations in drive
*   Unzip files in folder



In [ ]:
#Create folders for models and datasets
!mkdir "/content/drive/My Drive/RecipeNet" 
!mkdir "/content/drive/My Drive/RecipeNet/datasets" 
!mkdir "/content/drive/My Drive/RecipeNet/datasets/download" 
!mkdir "/content/drive/My Drive/RecipeNet/datasets/extracted" 
!mkdir "/content/drive/My Drive/RecipeNet/datasets/preprocessed" 
!mkdir "/content/drive/My Drive/RecipeNet/model" 
#Scarica dataset
#Trascina i 3 zip

#Unzip
"""
!unzip "/content/drive/My Drive/RecipeNet/datasets/download/new_right_train.zip" -d "/content/drive/My Drive/RecipeNet_g/datasets/extracted"
!unzip "/content/drive/My Drive/RecipeNet/datasets/download/archive.zip" -d "/content/drive/My Drive/RecipeNet/datasets/extracted"
!unzip "/content/drive/My Drive/RecipeNet/datasets/download/Ingredients101.zip" -d "/content/drive/My Drive/RecipeNet/datasets/extracted"

"""


#***Imports and Drive Mount***

In [ ]:
#Imports
import sys
import time
import os
import json
import numpy as np 
import pandas as pd 
from tqdm.notebook import tqdm, trange
import sys
import cv2
import torch
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from glob import glob
from sklearn.metrics import f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import torchvision.models as models
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict


#Cuda
device = torch.device("cuda")
torch.manual_seed(42) # try and make the results more reproducible
print(torch.cuda.get_device_name(0))

#Drive Mount
from google.colab import drive
drive.mount('/content/drive')


#***Project Parameters***


In [ ]:
params = {}

#DATASET KEYWORD
params["dataset"] = "Recipe5k"

#NEURAL NETWORK PARAMS
params["freezed_layers"] = 8

#GENERAL TRAINING PARAMS
params["epochs"] = 20
params["batch_size"] = 64
params["img_size"] = (384,384)

#FAST TRAINING SETTINGS
#1M: TR:67000 VAL:5400 TE:27000
#5k: TR: VAL: TE:
params["fast_training"] = True
params["train_df_size"] = 120
params["val_df_size"] = 120
params["test_df_size"] = 120


#DATASET SETTINGS
params["root"] = "/content/drive/My Drive/RecipeNet/"
if params["dataset"] == "Recipe5k":
    params["images_dir"] = params["root"] + "datasets/extracted/images/" 
    params["ingredients_per_class"] = params["root"] + "datasets/extracted/Ingredients101/Annotations/ingredients_simplified.txt" 
    params["classes"] = params["root"] + "datasets/extracted/Ingredients101/Annotations/classes.txt" 
    params["baseIngredients_dir"] = params["root"] + "datasets/extracted/Ingredients101/ingredients_simplification/baseIngredients.txt" 
    params["annotations_path"] = params["root"] + "datasets/extracted/Ingredients101/Annotations/"
elif params["dataset"] == "Recipe1M":
    params["images_dir"] = params["root"] + "datasets/extracted/new_right_train/" 
    params["annotations_path"] = params["root"] + "datasets/extracted/Annotations_1M/"
    params["ingredients_per_class"] = params["root"] + "datasets/extracted/Annotations_1M/ingr.txt" 
    params["classes"] = params["root"] + "datasets/extracted/Annotations_1M/newclasses.txt" 
    params["baseIngredients_dir"] = params["root"] + "datasets/extracted/Annotations_1M/baseIngredients.txt" 
    params["annotations_path"] = params["root"] + "datasets/extracted/Annotations_1M/"
else:
    print("Invalid Dataset Keyword, try with Recipe1M or Recipe5k")


#***Data extraction and preprocessing***


Dataset Loading

In [ ]:
#Ingredients x class
f = open(params["ingredients_per_class"], "r")
ingredients = f.read().split('\n')
f.close()

#Classes
f = open(params["classes"], "r")
classes = f.read().split('\n')
f.close()

#Ingredients
f = open(params["baseIngredients_dir"], "r")
base_ing = f.read().split('\n')
base_ing = base_ing[0].split(",")
f.close()

#train images
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/train_images.txt", "r")
train_images = f.read().split('\n')
f.close()
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/train_labels.txt", "r")
train_labels = f.read().split('\n')
f.close()

#validation images
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/val_images.txt", "r")
val_images = f.read().split('\n')
f.close()
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/val_labels.txt", "r")
val_labels = f.read().split('\n')
f.close()

#test images
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/test_images.txt", "r")
test_images = f.read().split('\n')
f.close()
f = open(params["root"] + "datasets/extracted/Ingredients101/Annotations/test_labels.txt", "r")
test_labels = f.read().split('\n')
f.close()


In [ ]:
#Ingredients x class
f = open(params["ingredients_per_class"], "r")
ingredients = f.read().split('\n')
f.close()

#Classes
f = open(params["classes"], "r")
classes = f.read().split('\n')
f.close()

#Ingredients
f = open(params["baseIngredients_dir"], "r")
base_ing = f.read().split('\n')
base_ing = base_ing[0].split(",")
f.close()

#train images
f = open(params["root"] + "datasets/extracted/Annotations_1M/train_imgs.txt", "r")
train_images = f.read().split('\n')
f.close()
f = open(params["root"] + "datasets/extracted/Annotations_1M/train_lab.txt", "r")
train_labels = f.read().split('\n')
f.close()

#validation images
f = open(params["root"] + "datasets/extracted/Annotations_1M/val_imgs.txt", "r")
val_images = f.read().split('\n')
f.close()
f = open(params["root"] + "datasets/extracted/Annotations_1M/val_lab.txt", "r")
val_labels = f.read().split('\n')
f.close()

#test images
f = open(params["root"] + "datasets/extracted/Annotations_1M/test_imgs.txt", "r")
test_images = f.read().split('\n')
f.close()
f = open(params["root"] + "datasets/extracted/Annotations_1M/test_lab.txt", "r")
test_labels = f.read().split('\n')
f.close()

Dataframes Creation

In [ ]:
#list of string in list of list of tokens
new_ingredients = [arr.split(",") for arr in ingredients]

#binary encode ingredients
mlb = MultiLabelBinarizer()
df = pd.DataFrame(mlb.fit_transform(new_ingredients),columns=mlb.classes_) 
df["target"] = classes
food_dict = df
train_images = [params["images_dir"] + s + ".jpg" for s in train_images]
all_img_df = pd.DataFrame({'path': train_images, 'class_id': train_labels})
val_images = [params["images_dir"] + s + ".jpg" for s in val_images]
val_img_df = pd.DataFrame({'path': val_images, 'class_id': val_labels})
test_images = [params["images_dir"] + s + ".jpg" for s in test_images]
test_img_df = pd.DataFrame({'path': test_images, 'class_id': test_labels})
all_img_df = all_img_df[:-1]
val_img_df = val_img_df[:-1]
test_img_df = test_img_df[:-1]



all_img_df['class_name'] = all_img_df['path'].map(lambda x: os.path.split(os.path.dirname(x))[-1])
val_img_df['class_name'] = val_img_df['path'].map(lambda x: os.path.split(os.path.dirname(x))[-1])
test_img_df['class_name'] = test_img_df['path'].map(lambda x: os.path.split(os.path.dirname(x))[-1])

#food_dict = food_dict.drop('', 1)


Train Dataframe

In [ ]:
#Dataframe for train images
new_data = []
for index, row in all_img_df.iterrows():
    #get binary encoding ingredients from lookup
    food = row["class_name"]
    path = row["path"]
    class_id = row["class_id"]
    binary_encod = food_dict.loc[food_dict["target"] == food]
    binary_encod["path"] = path
    binary_encod["class_id"] = class_id
    #print(binary_encod["class_id"])
    #print((list(binary_encod.columns.values)))
    #print(len(np.array(binary_encod)[0]))
    new_data.append(np.array(binary_encod)[0])



col_names = list(binary_encod.columns.values)
print(len(col_names))
train_df = pd.DataFrame(new_data, columns = col_names)


Validation Dataframe

In [ ]:
val_data = []
for index, row in val_img_df.iterrows():
    #get binary encoding ingredients from lookup
    food = row["class_name"]
    path = row["path"]
    class_id = row["class_id"]
    binary_encod = food_dict.loc[food_dict["target"] == food]
    binary_encod["path"] = path
    binary_encod["class_id"] = int(class_id)
    val_data.append(np.array(binary_encod)[0])
val_df = pd.DataFrame(val_data, columns = col_names)


Test Dataframe

In [ ]:
test_data = []
for index, row in test_img_df.iterrows():
    #get binary encoding ingredients from lookup
    food = row["class_name"]
    path = row["path"]
    class_id = row["class_id"]
    binary_encod = food_dict.loc[food_dict["target"] == food]
    binary_encod["path"] = path
    binary_encod["class_id"] = int(class_id)
    test_data.append(np.array(binary_encod)[0])


test_df = pd.DataFrame(test_data, columns = col_names)


#***DataGenerator***


In [ ]:
class DataWrapper(data.Dataset):
    ''' Data wrapper for pytorch's data loader function '''
    def __init__(self, image_df, resize):
        self.dataset = image_df
        self.resize = resize

    def __getitem__(self, index):
        c_row = self.dataset.iloc[index]
        target_arr = []
        for item in c_row[targets].values:
            target_arr.append(item)
        #print(target_arr)
        image_path, target = c_row['path'], torch.from_numpy(np.array(target_arr)).float()  #image and target
        #read as rgb image, resize and convert to range 0 to 1
        image = cv2.imread(image_path, 1)
        #print(image_path, image)
        if self.resize:
            image = cv2.resize(image, params["img_size"])/255.0 
        else:
            image = image/255.0
        image = (torch.from_numpy(image.transpose(2,0,1))).float() #NxCxHxW
        return image, target

    def __len__(self):
        return self.dataset.shape[0]

#***Trasfer Learning Model:ResNet***

In [ ]:
col_names = list(train_df.columns.values)
targets = col_names[:-3]

model = models.resnet50(pretrained=True)

output = len(targets)
model.fc = nn.Linear(model.fc.in_features, output)

layer = 0
for name, child in model.named_children():
    layer += 1
    if layer < params["freezed_layers"]:
        for name2, param in child.named_parameters():
            param.requires_grad = False

#***Training***

Define Hamming Score

In [ ]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

Training Settings

In [ ]:
if params["fast_training"] == True:
    train_df = train_df[:params["train_df_size"]]
    val_df = val_df[:params["val_df_size"]]
    test_df = test_df[:params["test_df_size"]]
            
print(len(val_df))
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())



Dataset Loaders

In [ ]:
train_dataset = DataWrapper(train_df, True)
train_loader = torch.utils.data.DataLoader(train_dataset,shuffle=True, batch_size=params["batch_size"], pin_memory=False)

val_dataset = DataWrapper(val_df, True)
val_loader = torch.utils.data.DataLoader(val_dataset,shuffle=True, batch_size=params["batch_size"], pin_memory=False)

test_dataset = DataWrapper(test_df, True)
test_loader = torch.utils.data.DataLoader(test_dataset,shuffle=True, batch_size=params["batch_size"], pin_memory=False)

#Definisco i plots
train_results = defaultdict(list)
train_iter, test_iter, best_acc = 0,0,0
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (10, 10))
ax1.set_title('Train Loss')
ax2.set_title('Train Accuracy')
ax3.set_title('Test Loss')
ax4.set_title('Test Accuracy')

#dizionario f1 scores
f1_scores = defaultdict(list)


Training

In [ ]:

for i in trange(params["epochs"], desc='Epochs'):
    print("Epoch ",i)
    ## Train Phase
    #Model switches to train phase
    model.train() 
    
    #azzera ris
    all_outputs = []
    all_targets = []

    # Running through all mini batches in the dataset
    count, loss_val, correct, total = train_iter, 0, 0, 0
    for img_data, target in tqdm(train_loader, desc='Training'):  

        #load batch  
        img_data, target = img_data.to(device), target.to(device)
        #FWD prop
        output = model(img_data) 

        #Cross entropy loss
        loss = criterion(output, target) 

        #current loss 
        c_loss = loss.data.item()

        #plot current loss
        ax1.plot(count, c_loss, 'r.')

        #accumulate loss for training phase
        loss_val += c_loss

        optimizer.zero_grad() #Zero out any cached gradients
        loss.backward() #Backward pass
        optimizer.step() #Update the weights

        #number of outputs, batch * labels
        total_batch = (target.size(0) * target.size(1))
        total += total_batch

        #output and targets
        output_data = torch.sigmoid(output)>=0.5
        target_data = (target==1.0)
        
        #carica i risultati e i target
        for arr1,arr2 in zip(output_data, target_data):
            all_outputs.append(list(arr1.cpu().numpy()))
            all_targets.append(list(arr2.cpu().numpy()))

        #accuracy
        c_acc = torch.sum((output_data == target_data.to(device)).to(torch.float)).item()
        
        #plot accuracy
        ax2.plot(count, c_acc/total_batch, 'r.')

        #accumula accuracy
        correct += c_acc

        #accumula numero di steps per epoch
        count +=1
        
    #prendi i risultati dell'ultimo training
    all_outputs = np.array(all_outputs)
    all_targets = np.array(all_targets)

    #Training Metrics
    #______________________________________________________________________________________________________
    f1score_samples = f1_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    f1score_macro = f1_score(y_true=all_targets, y_pred=all_outputs, average='macro')
    f1score_weighted = f1_score(y_true=all_targets, y_pred=all_outputs, average='weighted')
    recall = recall_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    prec = precision_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    hamming = hamming_score(y_true=all_targets, y_pred=all_outputs)
    
    f1_scores["samples_train"].append(f1score_samples)
    f1_scores["macro_train"].append(f1score_macro)
    f1_scores["weighted_train"].append(f1score_weighted)
    f1_scores["hamming_train"].append(hamming)
    #____________________________________________________________________________________________________________________
    #Training loss val è loss per image,
    #train_iter è count, numero di steps, images in training / batch size * epoch number
    #train_acc over training
    train_loss_val, train_iter, train_acc = loss_val/len(train_loader.dataset), count, correct/float(total)
    
    print("Training loss: ", train_loss_val, " train acc: ",train_acc)    
    ## Test Phase
    
    #Model switches to test phase
    model.eval()
    
    all_outputs = []
    all_targets = []
    #Running through all mini batches in the dataset
    count, correct, total, lost_val = test_iter, 0, 0, 0
    for img_data, target in tqdm(val_loader, desc='Validation'):
        img_data, target = img_data.to(device), target.to(device)
        print(img_data.size())
        output = model(img_data)
        loss = criterion(output, target) #Cross entropy loss
        c_loss = loss.data.item()
        ax3.plot(count, c_loss, 'b.')
        loss_val += c_loss
        #Compute accuracy
        #predicted = output.data.max(1)[1] #get index of max
        total_batch = (target.size(0) * target.size(1))
        total += total_batch
        output_data = torch.sigmoid(output)>=0.5
        target_data = (target==1.0)
        #print("Predictions: ", output_data)
        #print("Actual: ", target_data)
        for arr1,arr2 in zip(output_data, target_data):
            all_outputs.append(list(arr1.cpu().numpy()))
            all_targets.append(list(arr2.cpu().numpy()))
        c_acc = torch.sum((output_data == target_data.to(device)).to(torch.float)).item()
        ax4.plot(count, c_acc/total_batch, 'b.')
        correct += c_acc
        count += 1
    
    #print("Outputs: ", len(all_outputs), " x ", len(all_outputs[0]))
    #print("Targets: ", len(all_targets), " x ", len(all_targets[0]))
    
    #F1 Score
    all_outputs = np.array(all_outputs)
    all_targets = np.array(all_targets)
    f1score_samples = f1_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    f1score_macro = f1_score(y_true=all_targets, y_pred=all_outputs, average='macro')
    f1score_weighted = f1_score(y_true=all_targets, y_pred=all_outputs, average='weighted')
    recall = recall_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    prec = precision_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    hamming = hamming_score(y_true=all_targets, y_pred=all_outputs)
    
    f1_scores["samples_test"].append(f1score_samples)
    f1_scores["macro_test"].append(f1score_macro)
    f1_scores["weighted_test"].append(f1score_weighted)
    f1_scores["hamming_test"].append(hamming)
    
    #Accuracy over entire dataset
    test_acc, test_iter, test_loss_val = correct/float(total), count, loss_val/len(test_loader.dataset)
    print("Test set accuracy: ",test_acc)
    print("f1_scores", f1_scores)
    train_results['epoch'].append(i)
    train_results['train_loss'].append(train_loss_val)
    train_results['train_acc'].append(train_acc)
    train_results['train_iter'].append(train_iter)
    
    train_results['test_loss'].append(test_loss_val)
    train_results['test_acc'].append(test_acc)
    train_results['test_iter'].append(test_iter)
    
    #Save model with best accuracy
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), 'best_model.pth')
fig.savefig('train_curves.png')

#***Save***



In [ ]:
model_name = "test_recipe1m_13400_20_epochs"


PATH = "/content/drive/My Drive/RecipeNet/model/"+ model_name + ".pt"

torch.save({
            'epoch': params["epochs"],
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'f1_scores': f1_scores,
            'train_results': train_results 
            }, PATH)


#***Load***

In [ ]:
model_name = "test_recipe1m_13400_images_10_epochs"
model_name = "resnet_model_half_dataset"

#Definisco i plots
train_results = defaultdict(list)
train_iter, test_iter, best_acc = 0,0,0

#dizionario f1 scores
f1_scores = defaultdict(list)

PATH = "/content/drive/My Drive/RecipeNet/model/"+model_name+".pt"
checkpoint = torch.load(PATH)

model.load_state_dict(checkpoint['model_state_dict'])

optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

epoch = checkpoint['epoch']

f1_scores = checkpoint['f1_scores']

train_results = checkpoint['train_results']


#***Evaluation***

In [ ]:
#Model switches to test phase
model.eval()
"""
all_outputs = []
all_targets = []
#Running through all mini batches in the dataset
count, correct, total, lost_val = test_iter, 0, 0, 0
for img_data, target in tqdm(test_loader, desc='Testing'):
    img_data, target = img_data.to(device), target.to(device)
    output = model(img_data)
    loss = criterion(output, target) #Cross entropy loss
    c_loss = loss.data.item()
    ax3.plot(count, c_loss, 'b.')
    loss_val += c_loss
    #Compute accuracy
    #predicted = output.data.max(1)[1] #get index of max
    total_batch = (target.size(0) * target.size(1))
    total += total_batch
    output_data = torch.sigmoid(output)>=0.5
    target_data = (target==1.0)
    #print("Predictions: ", output_data)
    #print("Actual: ", target_data)
    for arr1,arr2 in zip(output_data, target_data):
        all_outputs.append(list(arr1.cpu().numpy()))
        all_targets.append(list(arr2.cpu().numpy()))
    c_acc = torch.sum((output_data == target_data.to(device)).to(torch.float)).item()
    correct += c_acc
    count += 1
"""
#F1 Score
all_outputs = np.array(all_outputs)
all_targets = np.array(all_targets)
f1score_samples = f1_score(y_true=all_targets, y_pred=all_outputs, average='samples')
f1score_macro = f1_score(y_true=all_targets, y_pred=all_outputs, average='macro')
f1score_weighted = f1_score(y_true=all_targets, y_pred=all_outputs, average='weighted')
recall = recall_score(y_true=all_targets, y_pred=all_outputs, average='samples')
prec = precision_score(y_true=all_targets, y_pred=all_outputs, average='samples')

hamming = hamming_score(y_true=all_targets, y_pred=all_outputs)

params["epochs"] = 19
print("__________________TRAIN RESULTS______________________")
print("F1_samples", f1_scores["samples_train"][params["epochs"]])
print("F1_macro", f1_scores["macro_train"][params["epochs"]])
print("F1_weighted", f1_scores["weighted_train"][params["epochs"]])
print("Hamming Score",f1_scores["hamming_train"][params["epochs"]])

print("__________________VAL RESULTS______________________")
print("F1_samples", f1_scores["samples_test"][params["epochs"]])
print("F1_macro", f1_scores["macro_test"][params["epochs"]])
print("F1_weighted", f1_scores["weighted_test"][params["epochs"]])
print("Hamming Score",f1_scores["hamming_test"][params["epochs"]])


print("__________________TEST RESULTS______________________")
print("F1_samples", f1score_samples)
print("F1_macro", f1score_macro)
print("F1_weighted", f1score_weighted)
print("Recall",recall)
print("Precision",prec)
print("Hamming Score",hamming)

In [ ]:
image_path = "/content/drive/My Drive/RecipeNet/pizza.jpg"
from google.colab.patches import cv2_imshow
#read as rgb image, resize and convert to range 0 to 1
image = cv2.imread(image_path, 1)
image = cv2.resize(image, params["img_size"])/255

print(len(new_ingredients), len(food_dict), len(col_names))
image = (torch.from_numpy(image.transpose(2,0,1))).float() 
image = image.unsqueeze(0)
image = image.to(device)

output = model(image) 

output_data = torch.sigmoid(output)>=0.5
"""
print(output_data)
print("output", len(output_data.cpu().numpy()[0]))
print("ingredients",len(col_names))
print(col_names)"""

for i, elem in enumerate(output_data.cpu().numpy()[0]):
    if elem:
        print(col_names[i])